# **CellphoneDB**

## cellphonedb

In [ ]:
# conda /disk213/xieqq/miniconda/bin/activate /disk213/xieqq/miniconda/envs/cpdb  #版本v5

import os
from cellphonedb.src.core.methods import cpdb_analysis_method
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

cpdb_file_path="/disk213/xieqq/JINHUA138/Single_cell_analysis.12.CellphoneDB/cellphonedb.zip"
meta_file_path="/disk213/xieqq/JINHUA138/Single_cell_analysis.12.CellphoneDB/8-6.Tissue_CM06/cellphonedb_meta.txt"
counts_file_path="/disk213/xieqq/JINHUA138/Single_cell_analysis.12.CellphoneDB/8-6.Tissue_CM06/Tissue_CM06.h5ad"
output_path="/disk213/xieqq/JINHUA138/Single_cell_analysis.12.CellphoneDB/8-6.Tissue_CM06/out"

cpdb_results = cpdb_statistical_analysis_method.call(
        cpdb_file_path = cpdb_file_path,
        meta_file_path = meta_file_path,
        counts_file_path = counts_file_path,
        counts_data = 'gene_name',
        score_interactions = True,
        output_path = output_path)

## R: plot

In [ ]:
library(dplyr)
library(tidyr)
library(stringr)
library(data.table)
library(tibble)
library(pheatmap)
library(circlize)
library(RColorBrewer)
library(ggplot2)

In [ ]:
setwd("/disk213/xieqq/JINHUA138/Single_cell_analysis.12.CellphoneDB")

In [ ]:
celltalker <- function(file_dir){
  
  file_means <- list.files(path=paste0("./",file_dir,"/out"), pattern="^statistical_analysis_means", full.names=TRUE)
  file_pvals <- list.files(path=paste0("./",file_dir,"/out"), pattern="^statistical_analysis_pvalues", full.names=TRUE)
  file_scores <- list.files(path=paste0("./",file_dir,"/out"), pattern="^statistical_analysis_interaction_scores", full.names=TRUE)
  
  means <- read.delim(file_means, check.names=FALSE) %>%
    mutate(gene_a=paste0(ifelse(str_starts(partner_a, "simple"), ",", partner_a), gene_a),
           gene_b=paste0(ifelse(str_starts(partner_b, "simple"), ",", partner_b), gene_b)) %>%
    dplyr::select(interacting_pair, gene_a, gene_b, matches("\\|")) %>%
    pivot_longer(cols=-c(interacting_pair, gene_a, gene_b), names_to="CtoC", values_to="means") %>%
    separate(CtoC, into=c("From", "To"), sep="[|]", remove=FALSE)
  
  pvals <- read.delim(file_pvals, check.names=FALSE) %>%
    mutate(gene_a=paste0(ifelse(str_starts(partner_a, "simple"), ",", partner_a), gene_a),
           gene_b=paste0(ifelse(str_starts(partner_b, "simple"), ",", partner_b), gene_b)) %>%
    dplyr::select(interacting_pair, gene_a, gene_b, matches("\\|")) %>%
    pivot_longer(cols=-c(interacting_pair, gene_a, gene_b), names_to="CtoC", values_to="pvals") %>%
    separate(CtoC, into=c("From", "To"), sep="[|]", remove=FALSE)
  
  scores <- read.delim(file_scores, check.names=FALSE) %>%
    mutate(gene_a=paste0(ifelse(str_starts(partner_a, "simple"), ",", partner_a), gene_a),
           gene_b=paste0(ifelse(str_starts(partner_b, "simple"), ",", partner_b), gene_b)) %>%
    dplyr::select(interacting_pair, gene_a, gene_b, matches("\\|")) %>%
    pivot_longer(cols=-c(interacting_pair, gene_a, gene_b), names_to="CtoC", values_to="scores") %>%
    separate(CtoC, into=c("From", "To"), sep="[|]", remove=FALSE)
  
  pldf <- left_join(means, pvals, by=c("interacting_pair","gene_a","gene_b","CtoC","From","To")) %>%
    left_join(., scores, by=c("interacting_pair","gene_a","gene_b","CtoC","From","To"))
  
  return(pldf)
}

celltalker_extract <- function(file_dir, target){
  
  file_name <- list.files(path=paste0("./",file_dir,"/out"), pattern="^statistical_analysis_significant", full.names=TRUE)
  
  pldf <- read.delim(file_name, check.names=FALSE) %>%
    mutate(gene_a=paste0(ifelse(str_starts(partner_a, "simple"), ",", partner_a), gene_a),
           gene_b=paste0(ifelse(str_starts(partner_b, "simple"), ",", partner_b), gene_b)) %>%
    dplyr::select(interacting_pair, gene_a, gene_b, matches("\\|")) %>%
    pivot_longer(cols=-c(interacting_pair, gene_a, gene_b), names_to="CtoC", values_to="means") %>%
    filter(!is.na(.data[["means"]])) %>%
    separate(CtoC, into=c("From", "To"), sep="[|]", remove=FALSE)
  
  pldf1 <- pldf %>% filter(To == target & From != target) %>% mutate(Info="Othercell_to_Target")
  pldf2 <- pldf %>% filter(From == target & To != target) %>% mutate(Info="Target_to_Othercell")
  pldf3 <- pldf %>% filter(From == target & To == target) %>% mutate(Info="Target_to_Target")
  
  pldf_out <- bind_rows(pldf1, pldf2, pldf3)
  
  pldf_out$Info <- gsub("Target", target, pldf_out$Info)
  
  return(pldf_out)
}

In [ ]:
pldf <- celltalker(file_dir="4.CellLineage")
write.csv(pldf,"./4.CellLineage/CellPhoneDB_Epithelial.csv",row.names=F)
pldf <- celltalker_extract(file_dir="4.CellLineage", target="Epithelial")
write.csv(pldf,"CellPhoneDB_filtered_Epithelial.csv",row.names=F)

In [ ]:
data_C <- read.csv("./4-2.Epithelial_Large/CellPhoneDB_filtered_Enterocytes_large.csv", check.names=FALSE) %>% top_n(10, means)
data_E <- read.csv("./4-1.Epithelial_Small/CellPhoneDB_filtered_Enterocytes_small.csv", check.names=FALSE) %>% top_n(10, means)

data <- data.frame(x="Enterocytes",y=paste0(data_E$CtoC,": ",data_E$interacting_pair),value=data_E$means) %>%
  rbind(.,data.frame(x="Colonocytes",y=paste0(data_C$CtoC,": ",data_C$interacting_pair),value=data_C$means)) %>% 
  arrange(desc(x), value) %>% 
  mutate(y=factor(y, levels=unique(y)))
write.csv(data,"Point_top10.csv",row.names=F)

plot <- ggplot(data, aes(x=x, y=y)) +
  geom_point(aes(size=value, color=x)) +
  scale_size_continuous(name="Value") +
  scale_color_discrete(name="X") +
  scale_color_manual(values=c("Enterocytes"="#004b23","Colonocytes"="#007200")) +
  labs(x=",", y=",") +
  theme_bw()+
  theme(axis.text.x=element_text(color="black",size=10),
        axis.text.y=element_text(color="black",size=10),
        axis.title.y=element_text(color="black",size=12),
        legend.text=element_text(color="black",size=10),
        legend.title=element_text(color="black",size=12),
        panel.grid.major=element_blank(),
        panel.grid.minor=element_blank())
ggsave(filename="Point_top10.pdf", plot=plot, width=6, height=6)

In [ ]:
data7 <- read.csv("./7-2.Tissue/CellPhoneDB_Intestine_Epithelial.csv", check.names=FALSE) %>% filter(pvals<0.05)
data8 <- read.csv("./8-3.Tissue_CM03/CellPhoneDB_Intestine_Epithelial.csv", check.names=FALSE) %>% filter(pvals<0.05)

data <- inner_join(data7,data8,by=c("interacting_pair","gene_a","gene_b","CtoC","From","To"),suffix=c(".Tissue", ".Tissue_CM03")) %>%
    mutate(regulated=case_when(means.Tissue>means.Tissue_CM03 & scores.Tissue>scores.Tissue_CM03 ~ "up",
                               means.Tissue<means.Tissue_CM03 & scores.Tissue<scores.Tissue_CM03 ~ "down",
                               TRUE ~ "none"))
write.csv(data,"CellphoneDB_circlize_all.csv",row.names=F)

cells <- c("Adipose_ASC","Adipose_Endothelial","Adipose_Immune","Adipose_Smooth_muscle","Intestine_Bcells",
           "Intestine_Endothelial","Intestine_Epithelial","Intestine_Mesenchymal","Intestine_Myeloid","Intestine_Plasma",
           "Intestine_T_ILC_NKcells","Liver_DC","Liver_Endothelial","Liver_Macrophage","Liver_Monocyte","Liver_NK_ILC",
           "Liver_T","Muscle_Endothelial","Muscle_FAPs","Muscle_Myofiber","Muscle_Satellite")

DATA1 <- data %>% filter(means.Tissue>1, means.Tissue_CM03>1, regulated=="down", From %in% cells, To %in% cells) %>% 
  group_by(From, To) %>% summarise(n=n(), .groups="drop") %>%
  pivot_wider(id_cols=everything(), names_from=To, values_from=n) %>%
  column_to_rownames(var="From")
write.csv(DATA1,"CellphoneDB_circlize_count.csv",row.names=T)

ego <- ggplot(DATA1, aes(x=From, y=To, size=n)) +
  geom_point(color="#b499c9") +
  scale_size(range=c(1,15)) +
  labs(x="Source cell", y="Target cell", size="Count")+
  theme_bw()+
  theme(axis.text.x=element_text(angle=90,color="black", size=10),
        axis.text.y=element_text(color="black", size=10),
        axis.title.x=element_text(color="black",size=12),
        axis.title.y=element_text(color="black",size=10),
        plot.title=element_text(color="black",size=12),
        legend.text=element_text(color="black",size=10),
        legend.title=element_text(color="black",size=10),
        panel.grid.major=element_blank(),
        panel.grid.minor=element_blank())
ggsave(filename="CellphoneDB_circlize_count.pdf", plot=ego, width=10, height=10)

DATA2 <- data %>% filter(means.Tissue>1, means.Tissue_CM03>1, regulated=="down", From %in% cells, To %in% cells) %>%
  group_by(CtoC) %>% 
  slice_max(order_by=means.Tissue_CM03, n=1, with_ties=T) %>% 
  slice_max(order_by=scores.Tissue_CM03, n=1) %>% 
  ungroup()
write.csv(DATA2,"CellphoneDB_circlize.csv",row.names=F)

cell_types <- unique(c(df$From, df$To))
get_colors <- function(cell_types, palette = "Set3") {
  n <- length(cell_types)
  max_colors <- brewer.pal.info[palette, "maxcolors"]
  if (n <= max_colors) {
    colors <- brewer.pal(n, palette)
  } else {
    colors <- colorRampPalette(brewer.pal(max_colors, palette))(n)
  }
  setNames(colors, cell_types)
}

col_map <- get_colors(cell_types, "Set3")

circos.clear()
circos.par(start.degree = 90, gap.degree = 4)
chordDiagram(
  df,
  grid.col=col_map, 
  transparency=0.3,
  annotationTrack="grid",
  preAllocateTracks=list(track.height = 0.1)
)

circos.trackPlotRegion(
  track.index=1,
  panel.fun=function(x, y) {
    sector.name=get.cell.meta.data("sector.index")
    circos.text(mean(get.cell.meta.data("xlim")), get.cell.meta.data("ylim")[1]+0.1,
                sector.name, facing="clockwise", niceFacing=TRUE, adj=c(0, 0.5))},
  bg.border=NA
)

In [ ]:
load("CellChatDB.human.rda") #CellChat

unique(DATA2$interacting_pair) %>% sort()
ip <- c("APOA1_ABCA1","APP_TNFRSF21","CCL5_CCR5","Cholesterol-Cholesterol-DHCR7_RORA",
        "Cholesterol-Cholesterol-LIPA_RORA","COL4A2_ADGRG6","COL4A2_ITGA1_ITGB1",
        "FN1_ITGA4_ITGB1","FN1_ITGA4_ITGB7","FN1_ITGAV_ITGB1","ICAM1_ITGAL_ITGB2",
        "ICAM1_ITGAM_ITGB2","ICAM1_ITGAL","ICAM2_ITGAL_ITGB2","ITGAV_ITGB1_ADGRE5",
        "JAG1_NOTCH2","LAMC1_ITGA2_ITGB1","LAMC1_ITGA6_ITGB1","PECAM1_CD38",
        "PTPRC_CD22","PTPRC_MRC1","THBS2_CD36")
db <- CellChatDB.human$interaction %>% filter(interaction_name %in% ip)
write.csv(db,"CellphoneDB_circlize_inf.csv",row.names=F)

In [ ]:
Cpd_pair <- c("APLP2_PIGR","APP_CD74","APOA1_ABCA1")

data2 <- read.csv("./4.CellLineage/CellPhoneDB_Epithelial.csv", check.names=FALSE) %>% 
  filter(interacting_pair %in% Cpd_pair) %>% filter(startsWith(From, "Epithelial")) %>% 
  mutate(To=paste0("Intestine_",To), interacting_pair=paste0(interacting_pair, ": Pig")) %>%
  select(interacting_pair,To,means,pvals)
data3 <- read.csv("./6.Human.Pig_Human/CellPhoneDB_Human_Epithelial.csv", check.names=FALSE) %>% 
  filter(interacting_pair %in% Cpd_pair) %>% filter(startsWith(From, "Epithelial")) %>% 
  mutate(To=paste0("Intestine_",To), interacting_pair=paste0(interacting_pair, ": Human")) %>%
  select(interacting_pair,To,means,pvals)
data4 <- read.csv("./5.Human.Pig/CellPhoneDB_Human.Pig_Epithelial.csv", check.names=FALSE) %>% 
  filter(interacting_pair %in% Cpd_pair) %>% filter(startsWith(From, "Epithelial")) %>% 
  mutate(To=paste0("Intestine_",To), interacting_pair=paste0(interacting_pair, ": Human+Pig")) %>%
  select(interacting_pair,To,means,pvals)
data7 <- read.csv("./7-2.Tissue/CellPhoneDB_Intestine_Epithelial.csv", check.names=FALSE) %>% 
  filter(interacting_pair %in% Cpd_pair) %>% filter(startsWith(From, "Intestine_Epithelial")) %>% 
  mutate(interacting_pair=paste0(interacting_pair, ": 7-2.Tissue")) %>%
  select(interacting_pair,To,means,pvals)
data8 <- read.csv("./8-3.Tissue_CM03/CellPhoneDB_Intestine_Epithelial.csv", check.names=FALSE) %>% 
  filter(interacting_pair %in% Cpd_pair) %>% filter(startsWith(From, "Intestine_Epithelial")) %>% 
  mutate(interacting_pair=paste0(interacting_pair, ": 8-3.Tissue_CM03")) %>%
  select(interacting_pair,To,means,pvals)
data9 <- read.csv("./9-3.Tissue_L9-28/CellPhoneDB_Intestine_Epithelial.csv", check.names=FALSE) %>% 
  filter(interacting_pair %in% Cpd_pair) %>% filter(startsWith(From, "Intestine_Epithelial")) %>% 
  mutate(interacting_pair=paste0(interacting_pair,": 9-3.Tissue_L9-28")) %>%
  select(interacting_pair,To,means,pvals)

DATA <- bind_rows(data2, data3, data4, data7, data8, data9)
write.csv(DATA,"Point_Cpd_pair.csv",row.names=F)

plot <- ggplot(DATA, aes(x=To, y=interacting_pair)) +
  geom_point(aes(size=means, color=pvals), na.rm = TRUE) +
  scale_size_continuous(name="Means", range=c(1, 8)) +
  scale_color_gradient(name="p-value",low="#d95b5b", high="#b499c9") +
  theme_bw()+
  theme(axis.text.x=element_text(angle=90,color="black",size=10),
        axis.text.y=element_text(color="black",size=10),
        axis.title.y=element_text(color="black",size=12),
        legend.text=element_text(color="black",size=10),
        legend.title=element_text(color="black",size=12),
        panel.grid.major=element_blank(),
        panel.grid.minor=element_blank())
ggsave(filename="Point_Cpd_pair.pdf", plot=plot, width=12, height=6)